# Download dependencies

In [ ]:
!pip3 install -q kaggle
!pip3 install -q kaggle-cli
!pip3 install transformers
!pip3 install datasets
!pip3 install torch
!pip3 install torchmetrics

     |████████████████████████████████| 74 kB 1.6 MB/s 
     |████████████████████████████████| 4.2 MB 38.0 MB/s 
     |████████████████████████████████| 112 kB 67.9 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
     |████████████████████████████████| 147 kB 79.5 MB/s 
  ERROR: Failed building wheel for lxml
    Running setup.py install for lxml ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-xlnowd1h/lxml_b3af18a135d1463995b8f32b5e08b7fe/setup.py'"'"'; __file__='"'"'/tmp/pip-install-xlnowd1h/lxml_b3af18a135d1463995b8f32b5e08b7fe/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-0vxdrnt7/install-record.txt --s

# Download dataset

In [ ]:
!kaggle datasets download -d amandam1/friends-scripts

  0% 0.00/1.75M [00:00<?, ?B/s]
100% 1.75M/1.75M [00:00<00:00, 168MB/s]


In [ ]:
!ls /content/friends-scripts.zip
!/usr/bin/yes | unzip friends-scripts.zip

/content/friends-scripts.zip
Archive:  friends-scripts.zip
  inflating: friends.csv             


# Process dataset

In [ ]:
import pandas as pd
orig_df = pd.read_csv('/content/friends.csv')

In [ ]:
orig_df

,Unnamed: 0,season_number,episode_number,episode_name,character,line
0,0,1,1,the one where monica gets a new roommate (the ...,monica,there's nothing to tell! he's just some guy i...
1,1,1,1,the one where monica gets a new roommate (the ...,joey,"c'mon, you're going out with the guy! there's..."
2,2,1,1,the one where monica gets a new roommate (the ...,chandler,"all right joey, be nice. so does he have a h..."
3,3,1,1,the one where monica gets a new roommate (the ...,phoebe,"wait, does he eat chalk?"
4,4,1,1,the one where monica gets a new roommate (the ...,phoebe,"just, 'cause, i don't want her to go through ..."
...,...,...,...,...,...,...
61251,61251,10,17,the last one part 1 and part 2,chandler,"oh, it's gonna be okay."
61252,61252,10,17,the last one part 1 and part 2,rachel,(crying) do you guys have to go to the new ho...
61253,61253,10,17,the last one part 1 and part 2,monica,we got some time.
61254,61254,10,17,the last one part 1 and part 2,rachel,"okay, should we get some coffee?"


In [ ]:
df = pd.read_csv('/content/friends.csv')[['character', 'line']]

In [ ]:
df.character.unique()

array(['monica', 'joey', 'chandler', 'phoebe', 'all', 'ross', 'rachel',
       'waitress', 'phoebe, ross, chandler, and\xa0joey', 'paul',
       'priest on tv', 'frannie', 'ross and rachel', 'customer', 'marsha',
       'carol', 'chandler and\xa0joey', 'mrs. geller', 'mr. geller',
       'susan', 'barry', 'robbie', 'bernice', 'dr. oberman', 'paula',
       'alan', 'the guys', 'lizzie', 'guys', 'leslie', 'kiki', 'joanne',
       'receptionist', 'pizza guy', 'monica and phoebe', 'both', 'kid',
       'angela', 'janice', 'woman', 'bob', 'aurora', 'director', 'jill',
       'monica, joey, and\xa0phoebe', 'mr. heckles', 'paolo', 'shelley',
       'aunt lillian', 'nurse', 'andrea', 'lowell', 'terry', 'guy',
       'girl', 'max', 'david', 'sandy', 'fun bobby', 'dick clark',
       'jay leno', 'mrs. bing', 'coma guy', 'joey, chandler, and ross',
       'carol and susan', "phoebe's assistant", 'joey and\xa0chandler',
       'joey and chandler', 'rachel and\xa0phoebe', 'girls', 'roger',
       '

In [ ]:
df = df[df.character.isin(['phoebe', 'ross', 'monica', 'rachel', 'joey', 'chandler'])]
df

,character,line
0,monica,there's nothing to tell! he's just some guy i...
1,joey,"c'mon, you're going out with the guy! there's..."
2,chandler,"all right joey, be nice. so does he have a h..."
3,phoebe,"wait, does he eat chalk?"
4,phoebe,"just, 'cause, i don't want her to go through ..."
...,...,...
61251,chandler,"oh, it's gonna be okay."
61252,rachel,(crying) do you guys have to go to the new ho...
61253,monica,we got some time.
61254,rachel,"okay, should we get some coffee?"


In [ ]:
df.groupby('character').count()

,line
character,
chandler,8356
joey,8179
monica,8276
phoebe,7397
rachel,9109
ross,9054


# One hot encode labels

In [ ]:
import torch

In [ ]:
characters = list(df.character.unique())
NUM_LABELS = len(characters)
def character_to_one_hot(row):
  return {
    'line' : row['line'],
    'labels' : torch.nn.functional.one_hot(
      torch.IntTensor([characters.index(row['character'])]).to(torch.int64),
      num_classes=NUM_LABELS
    )[0]
  }

In [ ]:
character_to_one_hot({'line' : 'what??', 'character' : 'phoebe'})

{'line': 'what??', 'labels': tensor([0, 0, 0, 1, 0, 0])}

# Training configs

In [ ]:
MODEL_NAME = 'distilbert-base-uncased'
DOWNSAMPLE_DATASET = False
LEARNING_RATE = 1e-4

# Prepare and tokenize dataset

In [ ]:
dataset.column_names

['character', 'line', '__index_level_0__']

In [ ]:
dataset = dataset.remove_columns(column_names=['__index_level_0__'])

In [ ]:
dataset = dataset.map(lambda row: character_to_one_hot(row), batched=False)

  0%|          | 0/50371 [00:00<?, ?ex/s]

In [ ]:
dataset.column_names

['character', 'line', 'labels']

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(entry):
  return tokenizer(entry["line"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.column_names

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/51 [00:00<?, ?ba/s]

['character', 'line', 'labels', 'input_ids', 'attention_mask']

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(column_names=['character', 'line'])

In [ ]:
tokenized_train_dataset, tokenized_test_dataset = tokenized_dataset.train_test_split(test_size=0.1).values()

In [ ]:
tokenized_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 50371
})

In [ ]:
tokenized_train_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 45333
})

# Model setup

In [ ]:
import torch
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
def freeze_weights(unit):
  unit.requires_grad = False
  unit.trainable = False

  for name, param in unit.named_parameters():
    param.requires_grad = False
    param.trainable = False

  for param in unit.parameters():
    param.requires_grad = False
    param.trainable = False

def simplify_model(model):
  model.pre_classifier = torch.nn.Identity()
  model.dropout = torch.nn.Identity()

In [ ]:
freeze_weights(model.distilbert)
freeze_weights(model.pre_classifier)
tokenizer.trainable = False
simplify_model(model)

# Training utils

In [ ]:
from transformers import EvalPrediction
from datasets import load_metric

dataset_metrics = {}
for metric_name in ["recall", "precision", "f1"]:
  dataset_metrics[metric_name] = load_metric(metric_name, "multiclass")

def one_hot_to_class_index(x):
  return torch.argmax(torch.Tensor(x), dim=-1, keepdim=False)

# batch, row, column
def compute_metrics(eval_pred: EvalPrediction):
  yhat_logits, y = eval_pred.predictions, eval_pred.label_ids

  assert type(yhat_logits) == np.ndarray

  y = one_hot_to_class_index(y)
  yhat = one_hot_to_class_index(
      torch.nn.functional.softmax(
          torch.Tensor(yhat_logits), dim=-1))

  for metric_name, metric in dataset_metrics.items():
    metric.add_batch(predictions=yhat, references=y)

  return {
    metric_name: value 
    for _, metric in dataset_metrics.items()
    for metric_name, value in metric.compute(average="weighted").items()
  }

class MultiClassTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs.logits

    assert self.model.config.num_labels == NUM_LABELS

    loss = torch.nn.CrossEntropyLoss()(logits, labels.float())
    return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
  output_dir="./results-all",
  learning_rate=LEARNING_RATE,
  per_device_train_batch_size=128,
  per_device_eval_batch_size=128,
  num_train_epochs=20,
  weight_decay=0.01,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  save_total_limit=3,
  logging_steps=1,
  load_best_model_at_end=True,
  metric_for_best_model="eval_f1"
)

trainer = MultiClassTrainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_train_dataset,
  eval_dataset=tokenized_test_dataset,
  compute_metrics=compute_metrics,
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


# Train model

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 45333
  Num Epochs = 20
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 7100


Epoch,Training Loss,Validation Loss,Recall,Precision,F1
1,1.778000,1.781272,0.203057,0.226590,0.116997
2,1.764400,1.776493,0.219730,0.209993,0.170707
3,1.713100,1.772248,0.226677,0.208071,0.183309
4,1.744700,1.769053,0.223104,0.273196,0.185099
5,1.789800,1.765828,0.238785,0.355934,0.210709
6,1.739800,1.764525,0.235411,0.282263,0.204256
7,1.809400,1.762877,0.240572,0.322912,0.215241
8,1.770900,1.761416,0.241366,0.311037,0.212379
9,1.754700,1.759762,0.244145,0.275451,0.218667
10,1.755900,1.759065,0.245534,0.271439,0.222326


***** Running Evaluation *****
  Num examples = 5038
  Batch size = 128
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./results-all/checkpoint-355
Configuration saved in ./results-all/checkpoint-355/config.json
Model weights saved in ./results-all/checkpoint-355/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5038
  Batch size = 128
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./results-all/checkpoint-710
Configuration 

TrainOutput(global_step=7100, training_loss=1.7606041187299808, metrics={'train_runtime': 17868.675, 'train_samples_per_second': 50.74, 'train_steps_per_second': 0.397, 'total_flos': 1.1846650713329664e+17, 'train_loss': 1.7606041187299808, 'epoch': 20.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5038
  Batch size = 128


{'eval_loss': 1.7549301385879517,
 'eval_recall': 0.24970226280269947,
 'eval_precision': 0.29009794130908206,
 'eval_f1': 0.22758177201605445,
 'eval_runtime': 86.2296,
 'eval_samples_per_second': 58.425,
 'eval_steps_per_second': 0.464,
 'epoch': 20.0}

In [ ]:
# del model
# del tokenizer
# del trainer

In [ ]:
# import torch
# torch.cuda.empty_cache()
# import gc
# gc.collect()

In [ ]:
# !nvidia-smi

# Test model

In [ ]:
tokens_info = tokenizer(["how are you doing?"], padding=True, truncation=True, return_tensors="pt")

In [ ]:
for key in tokens_info:
  print(key, tokens_info[key])

input_ids tensor([[ 101, 2129, 2024, 2017, 2725, 1029,  102]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1]])


In [ ]:
for i in range(6):
  print(tokenizer.decode(tokens_info['input_ids'][0][i]))

[CLS]
how
are
you
doing
?


In [ ]:
def tokens_info_to_gpu(tokens_info):
  gpu_tokens_info = {}
  for key, val in tokens_info.items():
    gpu_tokens_info[key] = val.cuda()
  return gpu_tokens_info

In [ ]:
with torch.no_grad():
  output = model(**tokens_info_to_gpu(tokens_info))

In [ ]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.2530, -0.3979, -0.1619, -0.3341,  0.0156, -0.0052]],
       device='cuda:0'), hidden_states=None, attentions=None)

In [ ]:
characters[output[0][0].argmax()]

'monica'